In [ ]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy

In [ ]:
from google.colab import drive #import csv in google drive
drive.mount("/drive")

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
import pandas as pd #import libraries
import numpy as np
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO
from datetime import datetime
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [ ]:
#functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
  blob_service_client = BlobServiceClient.from_connection_string(connect_str)
  blob_client = blob_service_client.get_blob_client(container = container_name, blob = blob_name)
  blob_client.upload_blob(data, overwrite = True)
  print(f"Upload to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
  blob_service_client = BlobServiceClient.from_connection_string(connect_str)
  blob_client = blob_service_client.get_blob_client(container = container_name, blob = blob_name)
  download_stream = blob_client.download_blob()
  return download_stream.readall()

In [ ]:
#extra functions
def week_of_month(dt):
  first_day = dt.replace(day = 1)
  dom = dt.first_dayadjusted_dom = dom + first_day.weekday()
  return int(ceil(adjusted_dom/7.0))

def get_week_of_year(date_str):
  #calculates the ISO week number of the year for a given date

  #parameters:
  #date_str (str): a date string in the format yyyy-mm-dd

  #returns:
  #int: ISO week number of the year

  #parse the input string to a datetime object
  date = datetime.strptime(date_str, '%Y-%m-%d')

  #get the iso calendar week number
  week_of_year = date.isocalendar()[1]

  return week_of_year

In [ ]:
#specify path to JSON config file
config_file_path = 'config.json'

#load the json configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

#print the configuration
connection_string = config["connectionString"]

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'crimedata'

#initalize the blobserviceclient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

#get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

crime_data_df = pd.DataFrame()

#list all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
  print(blob.name)
  blob_client = container_client.get_blob_client(blob = blob.name)
  blob_data = blob_client.download_blob()
  blob_content = blob_data.readall().decode('utf-8')
  df = pd.read_csv(StringIO(blob_content))
  #display the head of the dataframe
  print(df.shape)
  #only one csv so
  crime_data_df = df.copy()

crimedata.csv
(918443, 28)


In [ ]:
crime_data_df.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

In [ ]:
pd.set_option('display.max_columns', None)
crime_data_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,0,M,O,101.0,STREET,NaN,NaN,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,M,O,128.0,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,NaN,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,19,X,X,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,M,O,405.0,CLOTHING STORE,NaN,NaN,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,M,H,102.0,SIDEWALK,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [ ]:
crime_data_df['Premis Cd'] = crime_data_df['Premis Cd'].astype('Int64')
crime_data_df['DATE OCC'] = crime_data_df['DATE OCC'].str.split(' ').str[0]
crime_data_df['DATETIME OCC'] = crime_data_df['DATE OCC'] + ' ' + crime_data_df['TIME OCC'].astype(str) #combine two columns
crime_data_df['DATETIME OCC'] = pd.to_datetime(crime_data_df['DATETIME OCC'], errors='coerce') #force conversion
crime_data_df['DATETIME OCC'] = crime_data_df['DATETIME OCC'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M') if pd.notnull(x) and x.hour < 24 else x) #formatting datetime
crime_data_df.drop(columns = ['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'Rpt Dist No','Part 1-2','Mocodes', 'Weapon Used Cd', 'Weapon Desc','Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'Cross Street'])

,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Status,Status Desc,LOCATION,LAT,LON,DATETIME OCC
0,Wilshire,510,VEHICLE - STOLEN,0,M,O,101,STREET,AA,Adult Arrest,1900 S LONGWOOD AV,34.0375,-118.3506,2020-03-01 21:30
1,Central,330,BURGLARY FROM VEHICLE,47,M,O,128,BUS STOP/LAYOVER (ALSO QUERY 124),IC,Invest Cont,1000 S FLOWER ST,34.0444,-118.2628,2020-02-08 18:00
2,Southwest,480,BIKE - STOLEN,19,X,X,502,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",IC,Invest Cont,1400 W 37TH ST,34.0210,-118.3002,2020-11-04 17:00
3,Van Nuys,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),19,M,O,405,CLOTHING STORE,IC,Invest Cont,14000 RIVERSIDE DR,34.1576,-118.4387,2020-03-10 20:37
4,Hollywood,354,THEFT OF IDENTITY,28,M,H,102,SIDEWALK,IC,Invest Cont,1900 TRANSIENT,34.0944,-118.3277,2020-08-17 12:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918438,Foothill,888,TRESPASSING,79,M,O,501,SINGLE FAMILY DWELLING,AA,Adult Arrest,8700 SHARP AV,34.2278,-118.4051,2024-02-11 19:20
918439,Foothill,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",36,M,H,101,STREET,IC,Invest Cont,8700 LANKERSHIM BL,34.2271,-118.3878,2024-01-13 07:20
918440,Topanga,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),44,F,O,104,DRIVEWAY,IC,Invest Cont,22700 NAPA ST,34.2260,-118.6214,2024-02-17 23:30
918441,Olympic,624,BATTERY - SIMPLE ASSAULT,80,F,O,124,BUS STOP,IC,Invest Cont,HOBART BL,34.0690,-118.3054,2024-01-16 15:10


In [ ]:
#creating status dimension

#mapping dictionary
status_mapping = {
  'AA': 'Adult Arrest',
  'IC': 'Invest Cont',
  'JA': 'Juv Arrest',
  'AO': 'Adult Other',
  'JO': 'Juv Other',
  'CC': 'UNK'
}

unique_status_ids = crime_data_df["Status"].unique()
#convert array of unique values into a dataframe
unique_status_df = pd.DataFrame(unique_status_ids, columns = ['status_id'])

#applying th mapping to create new column with descriptions
unique_status_df['status_descr'] = unique_status_df['status_id'].map(status_mapping)
unique_status_df = unique_status_df[unique_status_df['status_id'] != '<NA>']
unique_status_df

,status_id,status_descr
0,AA,Adult Arrest
1,IC,Invest Cont
2,JA,Juv Arrest
3,AO,Adult Other
4,JO,Juv Other
5,CC,UNK


In [ ]:
#create crime dimension
crime_mapping = {
110: 'CRIMINAL HOMICIDE',
113: 'MANSLAUGHTER, NEGLIGENT',
121: 'RAPE, FORCIBLE',
122: 'RAPE, ATTEMPTED',
210: 'ROBBERY',
220: 'ATTEMPTED ROBBERY',
230: 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
231: 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER',
235: 'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT',
236: 'INTIMATE PARTNER - AGGRAVATED ASSAULT',
237: 'CHILD NEGLECT (SEE 300 W.I.C.)',
250: 'SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT',
251: 'SHOTS FIRED AT INHABITED DWELLING',
310: 'BURGLARY',
320: 'BURGLARY, ATTEMPTED',
330: 'BURGLARY FROM VEHICLE',
331: 'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)',
341: 'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD',
343: 'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)',
345: 'DISHONEST EMPLOYEE - GRAND THEFT',
347: 'GRAND THEFT / INSURANCE FRAUD',
349: 'GRAND THEFT / AUTO REPAIR',
350: 'THEFT, PERSON',
351: 'PURSE SNATCHING',
352: 'PICKPOCKET',
353: 'DRUNK ROLL',
354: 'THEFT OF IDENTITY',
410: 'BURGLARY FROM VEHICLE, ATTEMPTED',
420: 'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)',
421: 'THEFT FROM MOTOR VEHICLE - ATTEMPT',
432: 'BLOCKING DOOR INDUCTION CENTER',
433: 'DRIVING WITHOUT OWNER CONSENT (DWOC)',
434: 'FALSE IMPRISONMENT',
435: 'LYNCHING',
436: 'LYNCHING - ATTEMPTED',
437: 'RESISTING ARREST',
438: 'RECKLESS DRIVING',
439: 'FALSE POLICE REPORT',
440: 'THEFT PLAIN - PETTY ($950 & UNDER)',
441: 'THEFT PLAIN - ATTEMPT',
442: 'SHOPLIFTING - PETTY THEFT ($950 & UNDER)',
443: 'SHOPLIFTING - ATTEMPT',
444: 'DISHONEST EMPLOYEE - PETTY THEFT',
445: 'DISHONEST EMPLOYEE ATTEMPTED THEFT',
446: 'PETTY THEFT - AUTO REPAIR',
450: 'THEFT FROM PERSON - ATTEMPT',
451: 'PURSE SNATCHING - ATTEMPT',
452: 'PICKPOCKET, ATTEMPT',
470: 'TILL TAP - GRAND THEFT ($950.01 & OVER)',
471: 'TILL TAP - PETTY ($950 & UNDER)',
473: 'THEFT, COIN MACHINE - GRAND ($950.01 & OVER)',
474: 'THEFT, COIN MACHINE - PETTY ($950 & UNDER)',
475: 'THEFT, COIN MACHINE - ATTEMPT',
480: 'BIKE - STOLEN',
485: 'BIKE - ATTEMPTED STOLEN',
487: 'BOAT - STOLEN',
510: 'VEHICLE - STOLEN',
520: 'VEHICLE - ATTEMPT STOLEN',
522: 'VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, BIKES, ETC)',
622: 'BATTERY ON A FIREFIGHTER',
623: 'BATTERY POLICE (SIMPLE)',
624: 'BATTERY - SIMPLE ASSAULT',
625: 'OTHER ASSAULT',
626: 'INTIMATE PARTNER - SIMPLE ASSAULT',
627: 'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT',
647: 'THROWING OBJECT AT MOVING VEHICLE',
648: 'ARSON',
649: 'DOCUMENT FORGERY / STOLEN FELONY',
651: 'DOCUMENT WORTHLESS ($200.01 & OVER)',
652: 'DOCUMENT WORTHLESS ($200 & UNDER)',
653: 'CREDIT CARDS, FRAUD USE ($950.01 & OVER)',
654: 'CREDIT CARDS, FRAUD USE ($950 & UNDER',
660: 'COUNTERFEIT',
661: 'UNAUTHORIZED COMPUTER ACCESS',
662: 'BUNCO, GRAND THEFT',
664: 'BUNCO, PETTY THEFT',
666: 'BUNCO, ATTEMPT',
668: 'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)',
670: 'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)',
740: 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)',
745: 'VANDALISM - MISDEAMEANOR ($399 OR UNDER)',
753: 'DISCHARGE FIREARMS/SHOTS FIRED',
755: 'BOMB SCARE',
756: 'WEAPONS POSSESSION/BOMBING',
760: 'LEWD/LASCIVIOUS ACTS WITH CHILD',
761: 'BRANDISH WEAPON',
762: 'LEWD CONDUCT',
763: 'STALKING',
805: 'PIMPING',
806: 'PANDERING',
810: 'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ',
812: 'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)',
813: 'CHILD ANNOYING (17YRS & UNDER)',
814: 'CHILD PORNOGRAPHY',
815: 'SEXUAL PENETRATION W/FOREIGN OBJECT',
820: 'ORAL COPULATION',
821: 'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH',
822: 'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS',
830: 'INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)',
840: 'BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM',
845: 'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE',
850: 'INDECENT EXPOSURE',
860: 'BATTERY WITH SEXUAL CONTACT',
865: 'DRUGS, TO A MINOR',
870: 'CHILD ABANDONMENT',
880: 'DISRUPT SCHOOL',
882: 'INCITING A RIOT',
884: 'FAILURE TO DISPERSE',
886: 'DISTURBING THE PEACE',
888: 'TRESPASSING',
890: 'FAILURE TO YIELD',
900: 'VIOLATION OF COURT ORDER',
901: 'VIOLATION OF RESTRAINING ORDER',
902: 'VIOLATION OF TEMPORARY RESTRAINING ORDER',
903: 'CONTEMPT OF COURT',
904: 'FIREARMS EMERGENCY PROTECTIVE ORDER (FIREARMS EPO)',
906: 'FIREARMS RESTRAINING ORDER (FIREARMS RO)',
910: 'KIDNAPPING',
920: 'KIDNAPPING - GRAND ATTEMPT',
921: 'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE',
922: 'CHILD STEALING',
924: 'TELEPHONE PROPERTY - DAMAGE',
926: 'TRAIN WRECKING',
928: 'THREATENING PHONE CALLS/LETTERS',
930: 'CRIMINAL THREATS - NO WEAPON DISPLAYED',
931: 'REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)',
932: 'PEEPING TOM',
933: 'PROWLER',
940: 'EXTORTION',
942: 'BRIBERY',
943: 'CRUELTY TO ANIMALS',
944: 'CONSPIRACY',
946: 'OTHER MISCELLANEOUS CRIME',
948: 'BIGAMY',
949: 'ILLEGAL DUMPING',
950: 'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01',
951: 'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER',
954: 'CONTRIBUTING',
956: 'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD'

}

unique_crime_ids = crime_data_df["Crm Cd"].unique()
#convert array of unique values into a dataframe
unique_crime_df = pd.DataFrame(unique_crime_ids, columns = ['crime_id'])

#applying th mapping to create new column with descriptions
unique_crime_df['crm_cd_descr'] = unique_crime_df['crime_id'].map(crime_mapping)
unique_crime_df = unique_crime_df[unique_crime_df['crime_id'] != '<NA>']
unique_crime_df

,crime_id,crm_cd_descr
0,510,VEHICLE - STOLEN
1,330,BURGLARY FROM VEHICLE
2,480,BIKE - STOLEN
3,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER)
4,354,THEFT OF IDENTITY
...,...,...
134,830,INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)
135,432,BLOCKING DOOR INDUCTION CENTER
136,882,INCITING A RIOT
137,445,DISHONEST EMPLOYEE ATTEMPTED THEFT


In [ ]:
#create premis dimension
premis_mapping = {
101: "STREET",
102: "SIDEWALK",
103: "ALLEY",
104: "DRIVEWAY",
105: "PEDESTRIAN OVERCROSSING",
106: "TUNNEL",
107: "VACANT LOT",
108: "PARKING LOT",
109: "PARK/PLAYGROUND",
110: "FREEWAY",
111: "BUS-CHARTER/PRIVATE",
112: "BUS, SCHOOL, CHURCH",
113: "TRUCK, COMMERICAL",
114: "TAXI",
115: "AIRCRAFT",
116: "OTHER/OUTSIDE",
117: "BEACH",
118: "CONSTRUCTION SITE",
119: "PORCH, RESIDENTIAL",
120: "STORAGE SHED",
121: "YARD (RESIDENTIAL/BUSINESS)",
122: "VEHICLE, PASSENGER/TRUCK",
123: "PARKING UNDERGROUND/BUILDING",
124: "BUS STOP",
125: "PAY PHONE",
126: "TRAIN, OTHER THAN MTA (ALSO QUERY 809/810/811)",
127: "TRASH CAN/TRASH DUMPSTER",
128: "BUS STOP/LAYOVER (ALSO QUERY 124)",
129: "TERMINAL, OTHER THAN MTA",
135: "MTA PROPERTY OR PARKING LOT",
136: "OTHER INTERSTATE, CHARTER BUS",
137: "TRAM/STREETCAR(BOXLIKE WAG ON RAILS)*",
138: "STAIRWELL*",
139: "AMUSEMENT PARK*",
140: "BALCONY*",
141: "CEMETARY*",
142: "DRIVE THRU*",
143: "ESCALATOR*",
144: "GOLF COURSE*",
145: "MAIL BOX",
146: "PATIO*",
147: "POOL-PUBLIC/OUTDOOR OR INDOOR*",
148: "PUBLIC RESTROOM/OUTSIDE*",
149: "RIVER BED*",
150: "TOOL SHED*",
151: "TOW YARD*",
152: "UNDERPASS/BRIDGE*",
154: "TRAIN TRACKS",
155: "DAM/RESERVOIR",
156: "VALET",
157: "VEHICLE STORAGE LOT (CARS, TRUCKS, RV'S, BOATS, TRAILERS, ETC.)",
158: "TRANSIENT ENCAMPMENT",
201: "JEWELRY STORE",
202: "LIQUOR STORE",
203: "OTHER BUSINESS",
204: "MANUFACTURING COMPANY",
205: "GUN/SPORTING GOODS",
206: "TV/RADIO/APPLIANCE",
207: "BAR/COCKTAIL/NIGHTCLUB",
208: "AUTO SALES LOT",
209: "EQUIPMENT RENTAL",
210: "RESTAURANT/FAST FOOD",
211: "PAWN SHOP",
212: "TRANSPORTATION FACILITY (AIRPORT)",
213: "WAREHOUSE",
214: "BUS DEPOT/TERMINAL, OTHER THAN MTA",
215: "TRAIN DEPOT/TERMINAL, OTHER THAN MTA",
216: "SWAP MEET",
217: "AUTO REPAIR SHOP",
218: "BEAUTY/BARBER SHOP",
219: "CLEANER/LAUNDROMAT",
220: "NAIL SALON",
221: "PUBLIC STORAGE",
222: "LAUNDROMAT",
223: "VIDEO RENTAL STORE",
224: "SURPLUS SURVIVAL STORE",
225: "RECORD-CD MUSIC/COMPUTER GAME STORE",
227: "ABORTION CLINIC/ABORTION FACILITY*",
228: "BOWLING ALLEY*",
229: "CHECK CASHING*",
230: "DAY CARE/ADULTS*",
231: "DAY CARE/CHILDREN*",
232: "DELIVERY SERVICE (FED EX, UPS, COURIERS,COURIER SERVICE)*",
233: "TATTOO PARLOR*",
234: "OPTICAL OFFICE INSIDE STORE OR SUPERMARKET*",
235: "PHARMACY INSIDE STORE OR SUPERMARKET*",
236: "VISION CARE FACILITY*",
237: "HOSPICE",
238: "TELECOMMUNICATION FACILITY/LOCATION",
239: "MEDICAL MARIJUANA FACILITIES/BUSINESSES",
240: "DEPT OF DEFENSE FACILITY",
241: "ENERGY PLANT/FACILITY",
242: "DISCOUNT STORE (99 CENT,DOLLAR,ETC.",
243: "DIY CENTER (LOWE'S,HOME DEPOT,OSH,CONTRACTORS WAREHOUSE)",
244: "TOBACCO SHOP",
245: "CATERING/ICE CREAM TRUCK",
246: "METHADONE CLINIC",
247: "CAR WASH",
248: "CELL PHONE STORE",
249: "NURSERY/FLOWER SHOP",
250: "COMPUTER SERVICES/REPAIRS/SALES",
251: "RECYCLING CENTER",
252: "COFFEE SHOP (STARBUCKS, COFFEE BEAN, PEET'S, ETC.)",
253: "MORTUARY",
254: "STUDIO (FILM/PHOTOGRAPHIC/MUSIC)",
255: "AUTO DEALERSHIP (CHEVY, FORD, BMW, MERCEDES, ETC.)",
256: "",
301: "GAS STATION",
302: "FACTORY",
303: "OIL REFINERY",
304: "GARMENT MANUFACTURER",
305: "CHEMICAL STORAGE/MANUFACTURING PLANT",
401: "MINI-MART",
402: "MARKET",
403: "DRUG STORE",
404: "DEPARTMENT STORE",
405: "CLOTHING STORE",
406: "OTHER STORE",
407: "HARDWARE/BUILDING SUPPLY",
408: "AUTO SUPPLY STORE*",
409: "BEAUTY SUPPLY STORE",
410: "MEMBERSHIP STORE (COSTCO,SAMS CLUB)*",
411: "PET STORE",
412: "ELECTRONICS STORE (IE:RADIO SHACK, ETC.)",
413: "FURNITURE STORE",
414: "BOOK STORE",
415: "THE GROVE",
416: "THE BEVERLY CENTER",
417: "THE BEVERLY CONNECTION",
418: "",
501: "SINGLE FAMILY DWELLING",
502: "MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",
503: "HOTEL",
504: "OTHER RESIDENCE",
505: "MOTEL",
506: "ABANDONED BUILDING ABANDONED HOUSE",
507: "CONDOMINIUM/TOWNHOUSE",
508: "FRAT HOUSE/SORORITY/DORMITORY",
509: "MOBILE HOME/TRAILERS/CONSTRUCTION TRAILERS/RV'S/MOTORHOME",
510: "NURSING/CONVALESCENT/RETIREMENT HOME",
511: "PROJECT/TENEMENT/PUBLIC HOUSING",
512: "FOSTER HOME BOYS OR GIRLS*",
513: "ABATEMENT LOCATION",
514: "GROUP HOME",
515: "APARTMENT/CONDO COMMON LAUNDRY ROOM",
516: "SINGLE RESIDENCE OCCUPANCY (SRO'S) LOCATIONS",
517: "MISSIONS/SHELTERS",
518: "TRANSITIONAL HOUSING/HALFWAY HOUSE",
519: "SHORT-TERM VACATION RENTAL",
601: "SAVINGS & LOAN",
602: "BANK",
603: "CREDIT UNION",
604: "FINANCE COMPANY",
605: "AUTOMATED TELLER MACHINE (ATM)",
606: "DRIVE THRU BANKING (WINDOW)*",
607: "BANKING INSIDE MARKET-STORE *",
608: "BANK DROP BOX/MONEY DROP-OUTSIDE OF BANK*",
701: "HOSPITAL",
702: "OFFICE BUILDING/OFFICE",
703: "ENTERTAINMENT/COMEDY CLUB (OTHER)",
704: "ELEMENTARY SCHOOL",
705: "SLIPS/DOCK/MARINA/BOAT",
706: "SEX ORIENTED/BOOK STORE/STRIP CLUB/GENTLEMAN'S CLUB",
707: "GARAGE/CARPORT",
708: "CHURCH/CHAPEL (CHANGED 03-03 FROM CHURCH/TEMPLE)",
709: "PUBLIC RESTROOM(INDOORS-INSIDE)",
710: "OTHER PREMISE",
711: "ARCADE,GAME ROOM/VIDEO GAMES (EXAMPLE CHUCKIE CHEESE)*",
712: "COLISEUM",
713: "CONVENTION CENTER",
714: "DODGER STADIUM",
715: "HORSE RACING/SANTA ANITA PARK*",
716: "THEATRE/MOVIE",
717: "HEALTH SPA/GYM",
718: "MASSAGE PARLOR",
719: "MEDICAL/DENTAL OFFICES",
720: "JUNIOR HIGH SCHOOL",
721: "HIGH SCHOOL",
722: "COLLEGE/JUNIOR COLLEGE/UNIVERSITY",
723: "PRIVATE SCHOOL/PRESCHOOL",
724: "TRADE SCHOOL (MEDICAL-TECHNICAL-BUSINESS)*",
725: "GOVERNMENT FACILITY (FEDERAL,STATE, COUNTY & CITY)",
726: "POLICE FACILITY",
727: "SHOPPING MALL (COMMON AREA)",
728: "ELEVATOR",
729: "SPECIALTY SCHOOL/OTHER",
730: "SYNAGOGUE/TEMPLE",
731: "MOSQUE*",
732: "POST OFFICE",
733: "BAR/SPORTS BAR (OPEN DAY & NIGHT)",
734: "HOCKEY RINK/ICE HOCKEY",
735: "NIGHT CLUB (OPEN EVENINGS ONLY)",
736: "SKATEBOARD FACILITY/SKATEBOARD PARK*",
737: "SKATING RINK*",
738: "LIBRARY",
739: "SPORTS ARENA",
740: "OTHER PLACE OF WORSHIP",
741: "STAPLES CENTER *",
742: "SPORTS VENUE, OTHER",
743: "MASS GATHERING LOCATION",
744: "HIGH-RISE BUILDING",
745: "WATER FACILITY",
746: "SEWAGE FACILITY/PIPE",
748: "CULTURAL SIGNIFICANCE/MONUMENT",
750: "CYBERSPACE",
751: "WEBSITE",
752: "FIRE STATION",
753: "DETENTION/JAIL FACILITY",
754: "MUSEUM",
755: "VETERINARIAN/ANIMAL HOSPITAL",
756: "MUSCLE BEACH",
757: "BASKETBALL COURTS",
758: "HANDBALL COURTS",
801: "MTA BUS",
802: "MUNICIPAL BUS LINE INCLUDES LADOT/DASH",
803: "RETIRED (DUPLICATE) DO NOT USE THIS CODE",
804: "GREYHOUND OR INTERSTATE BUS",
805: "RETIRED (DUPLICATE) DO NOT USE THIS CODE",
809: "METROLINK TRAIN",
810: "AMTRAK TRAIN",
811: "OTHER RR TRAIN (UNION PAC, SANTE FE ETC",
834: "LA UNION STATION (NOT LINE SPECIFIC)",
835: "7TH AND METRO CENTER (NOT LINE SPECIFIC)",
836: "HARBOR FRWY STATION (NOT LINE SPECIFIC)",
868: "MTA - ORANGE LINE - NORTH HOLLYWOOD",
869: "MTA - ORANGE LINE - LAUREL CANYON",
870: "MTA - ORANGE LINE - VALLEY COLLEGE",
871: "MTA - ORANGE LINE - WOODMAN",
872: "MTA - ORANGE LINE - VAN NUYS",
873: "MTA - ORANGE LINE - SEPULVEDA",
874: "MTA - ORANGE LINE - WOODLEY",
875: "MTA - ORANGE LINE - BALBOA",
876: "MTA - ORANGE LINE - RESEDA",
877: "MTA - ORANGE LINE - TAMPA",
878: "MTA - ORANGE LINE - PIERCE COLLEGE",
879: "MTA - ORANGE LINE - DE SOTO",
880: "MTA - ORANGE LINE - CANOGA",
882: "MTA - ORANGE LINE - SHERMAN WAY",
883: "MTA - ORANGE LINE - ROSCOE",
884: "MTA - ORANGE LINE - NORDHOFF",
885: "MTA - ORANGE LINE - CHATSWORTH",
889: "MTA - SILVER LINE - LAC/USC MEDICAL CENTER",
890: "MTA - SILVER LINE - UNION STATION",
891: "MTA - SILVER LINE - DOWNTOWN STREET STOPS",
892: "MTA - SILVER LINE - 37TH ST/USC",
893: "MTA - SILVER LINE - SLAUSON",
894: "MTA - SILVER LINE - MANCHESTER",
895: "MTA - SILVER LINE - HARBOR FWY",
896: "MTA - SILVER LINE - ROSECRANS",
897: "MTA - SILVER LINE - HARBOR GATEWAY TRANSIT CTR",
898: "MTA - SILVER LINE - PACIFIC COAST HWY",
900: "MTA - RED LINE - UNION STATION",
901: "MTA - RED LINE - CIVIC CENTER/GRAND PARK",
902: "MTA - RED LINE - PERSHING SQUARE",
903: "MTA - RED LINE - 7TH AND METRO CENTER",
904: "MTA - RED LINE - WESTLAKE/MACARTHUR PARK",
905: "MTA - RED LINE - WILSHIRE/VERMONT",
906: "MTA - RED LINE - VERMONT/BEVERLY",
907: "MTA - RED LINE - VERMONT/SANTA MONICA",
908: "MTA - RED LINE - VERMONT/SUNSET",
909: "MTA - RED LINE - HOLLYWOOD/WESTERN",
910: "MTA - RED LINE - HOLLYWOOD/VINE",
911: "MTA - RED LINE - HOLLYWOOD/HIGHLAND",
912: "MTA - RED LINE - UNIVERSAL CITY/STUDIO CITY",
913: "MTA - RED LINE - NORTH HOLLYWOOD",
915: "MTA - PURPLE LINE - UNION STATION",
916: "MTA - PURPLE LINE - CIVIC CENTER/GRAND PARK",
917: "MTA - PURPLE LINE - PERSHING SQUARE",
918: "MTA - PURPLE LINE - 7TH AND METRO CENTER",
919: "MTA - PURPLE LINE - WESTLAKE/MACARTHUR PARK",
920: "MTA - PURPLE LINE - WILSHIRE/VERMONT",
921: "MTA - PURPLE LINE - WILSHIRE/NORMANDIE",
922: "MTA - PURPLE LINE - WILSHIRE/WESTERN",
931: "MTA - BLUE LINE - 7TH AND METRO CENTER",
932: "MTA - BLUE LINE - PICO",
933: "MTA - BLUE LINE - GRAND/LATTC",
934: "MTA - BLUE LINE - SAN PEDRO",
935: "MTA - BLUE LINE - WASHINGTON",
936: "MTA - BLUE LINE - VERNON",
937: "MTA - BLUE LINE - 103RD/WATTS TOWERS",
940: "MTA - EXPO LINE - 7TH AND METRO CENTER",
941: "MTA - EXPO LINE - PICO",
942: "MTA - EXPO LINE - LATTC/ORTHO INSTITUTE",
943: "MTA - EXPO LINE - JEFFERSON/USC",
944: "MTA - EXPO LINE - EXPO PARK/USC",
945: "MTA - EXPO LINE - EXPO/VERMONT",
946: "MTA - EXPO LINE - EXPO/WESTERN",
947: "MTA - EXPO LINE - EXPO/CRENSHAW",
948: "MTA - EXPO LINE - FARMDALE",
949: "MTA - EXPO LINE - EXPO/LA BREA",
950: "MTA - EXPO LINE - LA CIENEGA/JEFFERSON",
951: "MTA - EXPO LINE - PALMS",
952: "MTA - EXPO LINE - WESTWOOD/RANCHO PARK",
953: "MTA - EXPO LINE - EXPO/SEPULVEDA",
954: "MTA - EXPO LINE - EXPO/BUNDY",
956: "MTA - GREEN LINE - AVALON",
957: "MTA - GREEN LINE - HARBOR FWY",
958: "MTA - GREEN LINE - AVIATION/LAX",
961: "MTA - GOLD LINE - INDIANA",
962: "MTA - GOLD LINE - SOTO",
963: "MTA - GOLD LINE - MARIACHI PLAZA",
964: "MTA - GOLD LINE - PICO/ALISO",
965: "MTA - GOLD LINE - LITTLE TOKYO/ARTS DISTRICT",
966: "MTA - GOLD LINE - UNION STATION",
967: "MTA - GOLD LINE - CHINATOWN",
968: "MTA - GOLD LINE - LINCOLN/CYPRESS",
969: "MTA - GOLD LINE - HERITAGE SQ",
970: "MTA - GOLD LINE - SOUTHWEST MUSEUM",
971: "MTA - GOLD LINE - HIGHLAND PARK",
}

unique_premis_ids = crime_data_df["Premis Cd"].unique()
#convert array of unique values into a dataframe
unique_premis_df = pd.DataFrame(unique_premis_ids, columns = ['premis_id'])

#applying th mapping to create new column with descriptions
unique_premis_df['premis_desc'] = unique_premis_df['premis_id'].map(premis_mapping)
unique_premis_df = unique_premis_df[unique_premis_df['premis_id'] != '<NA>']
unique_premis_df

,premis_id,premis_desc
0,101,STREET
1,128,BUS STOP/LAYOVER (ALSO QUERY 124)
2,502,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
3,405,CLOTHING STORE
4,102,SIDEWALK
...,...,...
310,836,HARBOR FRWY STATION (NOT LINE SPECIFIC)
311,972,NaN
312,973,NaN
313,976,NaN


In [ ]:
victim_lookup_df = pd.read_csv('data/lookupvict.csv')
victim_lookup_df.head()

,Vict Age,Vict Sex,Vict Descent
0,0,M,O
1,47,M,O
2,19,X,X
3,19,M,O
4,28,M,H


In [ ]:
#create victim dimension
rename_vict_mapping = {
    'Vict Age': 'vict_age',
    'Vict Sex': 'vict_sex',
    'Vict Descent': 'vict_descent'
}
victim_lookup_df = victim_lookup_df.rename(columns = rename_vict_mapping)
victim_lookup_df['victim_id'] = range(1, len(victim_lookup_df) + 1)
new_vict_order = ['victim_id', 'vict_age', 'vict_sex', 'vict_descent']
victim_lookup_df = victim_lookup_df[new_vict_order]
victim_lookup_df.head()

,victim_id,vict_age,vict_sex,vict_descent
0,1,0,M,O
1,2,47,M,O
2,3,19,X,X
3,4,19,M,O
4,5,28,M,H


In [ ]:
location_lookup_df = pd.read_csv('data/lookuplocation.csv')
location_lookup_df.head()

,AREA NAME,LOCATION,LAT,LON
0,Wilshire,1900 S LONGWOOD AV,34.0375,-118.3506
1,Central,1000 S FLOWER ST,34.0444,-118.2628
2,Southwest,1400 W 37TH ST,34.0210,-118.3002
3,Van Nuys,14000 RIVERSIDE DR,34.1576,-118.4387
4,Hollywood,1900 TRANSIENT,34.0944,-118.3277


In [ ]:
#create location dimension
rename_loc_mapping = {
    'AREA NAME': 'area_name',
    'LOCATION': 'location',
    'LAT': 'latitude',
    'LON': 'longitude'
}
location_lookup_df = location_lookup_df.rename(columns = rename_loc_mapping)
location_lookup_df['location_id'] = range(1, len(location_lookup_df) + 1)
newloc_order = ['location_id', 'area_name', 'location', 'latitude', 'longitude']
location_lookup_df = location_lookup_df[newloc_order]
location_lookup_df.head()

,location_id,area_name,location,latitude,longitude
0,1,Wilshire,1900 S LONGWOOD AV,34.0375,-118.3506
1,2,Central,1000 S FLOWER ST,34.0444,-118.2628
2,3,Southwest,1400 W 37TH ST,34.0210,-118.3002
3,4,Van Nuys,14000 RIVERSIDE DR,34.1576,-118.4387
4,5,Hollywood,1900 TRANSIENT,34.0944,-118.3277


In [ ]:
#create datetime dimension

def week_of_month(dt):
  year = dt.year
  month = dt.month
  day = dt.day

  cal = calendar.monthcalendar(year, month)
  week_number = (day-1) // 7 + 1
  return week_number

start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2024-03-19')
#create dataframe for date dimension
date_dimension = pd.DataFrame({'datetime': pd.date_range(start_date, end_date, freq='H')})

date_dimension.head(25)

#extract attributes
date_dimension['year_number'] = date_dimension['datetime'].dt.year
date_dimension['quarter_number'] = date_dimension['datetime'].dt.quarter
date_dimension['month_number'] = date_dimension['datetime'].dt.month
date_dimension['monthname'] = date_dimension['datetime'].dt.strftime('%B')
date_dimension['day_number'] = date_dimension['datetime'].dt.day
date_dimension['dayname'] = date_dimension['datetime'].dt.strftime('%A')
date_dimension['hour_number'] = date_dimension['datetime'].dt.hour
date_dimension['date_iso_format'] = date_dimension['datetime'].apply(lambda x: x.isoformat())
date_dimension['datetime_id'] = date_dimension['datetime'].dt.strftime('%Y%m%d%H')

#add week of the month and week of the year
date_dimension['weekofmonth'] = date_dimension['datetime'].apply(week_of_month)
date_dimension['weekofyear'] = date_dimension['datetime'].dt.strftime('%U')

new_order = ['datetime_id', 'date_iso_format', 'year_number', 'quarter_number', 'month_number', 'day_number', 'hour_number', 'monthname', 'dayname', 'weekofyear', 'weekofmonth']
date_dimension = date_dimension[new_order]

date_dimension.head(25)

,datetime_id,date_iso_format,year_number,quarter_number,month_number,day_number,hour_number,monthname,dayname,weekofyear,weekofmonth
0,2020010100,2020-01-01T00:00:00,2020,1,1,1,0,January,Wednesday,00,1
1,2020010101,2020-01-01T01:00:00,2020,1,1,1,1,January,Wednesday,00,1
2,2020010102,2020-01-01T02:00:00,2020,1,1,1,2,January,Wednesday,00,1
3,2020010103,2020-01-01T03:00:00,2020,1,1,1,3,January,Wednesday,00,1
4,2020010104,2020-01-01T04:00:00,2020,1,1,1,4,January,Wednesday,00,1
5,2020010105,2020-01-01T05:00:00,2020,1,1,1,5,January,Wednesday,00,1
6,2020010106,2020-01-01T06:00:00,2020,1,1,1,6,January,Wednesday,00,1
7,2020010107,2020-01-01T07:00:00,2020,1,1,1,7,January,Wednesday,00,1
8,2020010108,2020-01-01T08:00:00,2020,1,1,1,8,January,Wednesday,00,1
9,2020010109,2020-01-01T09:00:00,2020,1,1,1,9,January,Wednesday,00,1


In [ ]:
crime_data_df['datetime_occ'] = pd.to_datetime(crime_data_df['DATETIME OCC']).dt.strftime('%Y%m%d%H')

new_column_names = {
    "Crm Cd": "crime_id",
    "Premis Cd": "premis_id",
    "Status": "status_id",
}
crime_data_df = crime_data_df.rename(columns=new_column_names)
crime_data_df['fact_id'] = range(1, len(crime_data_df) + 1)
crime_data_df['victim_id'] = range(1, len(crime_data_df) + 1)
crime_data_df['location_id'] = range(1, len(crime_data_df) + 1)
crime_data_df.drop(columns = ['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'Rpt Dist No','Part 1-2','Mocodes', 'Weapon Used Cd', 'Weapon Desc','Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'Cross Street', 'Date Rptd', 'DATETIME OCC'])
new_order = ['fact_id', 'crime_id', 'premis_id', 'victim_id', 'location_id', 'status_id', 'datetime_occ']
crime_data_df = crime_data_df[new_order]
crime_data_df.head()

,fact_id,crime_id,premis_id,victim_id,location_id,status_id,datetime_occ
0,1,510,101,1,1,AA,2020030121
1,2,330,128,2,2,IC,2020020818
2,3,480,502,3,3,IC,2020110417
3,4,343,405,4,4,IC,2020031020
4,5,354,102,5,5,IC,2020081712


In [ ]:
#Database connection URL
#replace the placeholders with your actual database credentials
database_url = ''

#create sqlalchemy engine
engine = create_engine(database_url)

In [ ]:
date_dimension.to_sql('dim_datetime', con = engine, if_exists = 'append', index = False)
crime_data_df.to_sql('facts_incident', con = engine, if_exists = 'append', index = False)
unique_crime_df.to_sql('dim_crime', con = engine, if_exists = 'append', index = False) #append to database/model
unique_premis_df.to_sql('dim_premise', con = engine, if_exists = 'append', index = False)
victim_lookup_df.to_sql('dim_victim', con = engine, if_exists = 'append', index = False)
location_lookup_df.to_sql('dim_location', con = engine, if_exists = 'append', index = False)
unique_status_df.to_sql('dim_status', con = engine, if_exists = 'append', index = False)

In [ ]:
#create csv
#unique_crime_df.to_csv('dim_crime', index = False)
#unique_premis_df.to_csv('dim_premise', index = False)
#victim_lookup_df.to_csv('dim_victim', index = False)
#location_lookup_df.to_csv('dim_location', index = False)
#unique_status_df.to_csv('dim_status', index = False)
#date_dimension.to_csv('dim_datetime', index = False)
#crime_data_df.to_csv('facts_incident', index = False)